In [1]:
from ALLCools.mcds import RegionDS
from ALLCools.dmr import call_dms, call_dmr
import glob as glob


In [2]:
mc_bulk_dir = '/data2st1/junyi/merged_allcs/PFC/'
allc_paths = glob.glob(f'{mc_bulk_dir}/*.tsv.gz')

In [3]:
allc_table= {}
for allc_path in allc_paths:
    samplename = allc_path.split('.')[0]
    samplepath = allc_path
    allc_table[samplename] = samplepath

In [4]:
allc_table

{'/data2st1/junyi/merged_allcs/PFC/OPC-Oligo': '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Astro-Epen': '/data2st1/junyi/merged_allcs/PFC/Astro-Epen.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Vascular': '/data2st1/junyi/merged_allcs/PFC/Vascular.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Neuron': '/data2st1/junyi/merged_allcs/PFC/Neuron.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Immune': '/data2st1/junyi/merged_allcs/PFC/Immune.tsv.gz'}

In [5]:
# make a dict, key is sample name, value is allc path
# import pathlib
# allc_table = {
#     allc_path.name.split('.')[0]: str(allc_path)
#     for allc_path in pathlib.Path(mc_bulk_dir).glob(
#         f'{mc_bulk_dir}/*.tsv.gz')
# }
samples = list(allc_table.keys())
allc_paths = list(allc_table.values())

chrom_size_path = '/home/junyichen/code/whole_mouse_brain/wmb/files/mm10.main.chrom.sizes'
output_dir = '/data2st1/junyi/merged_allcs/PFC/test_PFC'


In [179]:
call_dms(
    output_dir=output_dir,
    allc_paths=allc_paths,
    samples=samples,
    chrom_size_path=chrom_size_path,
    cpu=45,
    max_row_count=50,
    n_permute=3000,
    min_pvalue=0.01,
    # here we just calculate some small regions for demo
    # do not provide region parameter if you want to run DMR calling for the whole genome
    # This parameter can also be used for call DMR/DMS in specific region of interest
    region=['chr6:103638949-103659450'])


RMS tests for 7309 sites.


In [180]:
call_dmr(output_dir=output_dir,
         p_value_cutoff=0.001,
         frac_delta_cutoff=0.3,
         max_dist=250,
         residual_quantile=0.7,
         corr_cutoff=0.3,
         chrom = 'chr6',
         cpu=30)

In [181]:
dmr_ds = RegionDS.open(output_dir)
dmr_ds


Using dmr as region_dim


<xarray.RegionDS>
Dimensions:      (count_type: 2, dmr: 12, sample: 5)
Coordinates:
  * count_type   (count_type) <U3 'mc' 'cov'
  * dmr          (dmr) <U7 'chr6-0' 'chr6-1' 'chr6-2' ... 'chr6-10' 'chr6-11'
    dmr_chrom    (dmr) <U4 dask.array<chunksize=(2,), meta=np.ndarray>
    dmr_end      (dmr) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    dmr_length   (dmr) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    dmr_ndms     (dmr) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    dmr_start    (dmr) int64 dask.array<chunksize=(2,), meta=np.ndarray>
  * sample       (sample) <U43 '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo' ....
Data variables:
    dmr_da       (sample, dmr, count_type) uint32 dask.array<chunksize=(1, 2, 1), meta=np.ndarray>
    dmr_da_frac  (sample, dmr) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    dmr_state    (sample, dmr) int8 dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    region_dim:          dmr
    region_ds_location:  /data2st1/junyi/merged_allcs/PFC/test_PFC
    chrom_size_path:     /data2st1/junyi/merged_allcs/PFC/test_PFC/chrom_size...

In [182]:
dmr_ds['dmr_da_frac'].to_numpy()[3]

array([0.98      , 0.6393443 , 0.6883117 , 0.43421054, 0.42477876,
       0.5070422 , 0.4651163 , 0.30666667, 0.3409091 , 0.48333332,
       0.30769232, 0.6909091 ], dtype=float32)

In [ ]:
dmr_ds['dmr_da_frac'].to_numpy()[1]

array([[0.33333334, 0.18181819, 0.5555556 , 0.        , 0.8       ,
        0.        , 0.9444444 , 0.8947368 , 0.9411765 , 1.        ,
        0.95      , 0.43333334],
       [0.9       , 0.        , 0.33333334, 0.07692308, 0.90909094,
        0.15384616, 0.8888889 , 0.96      , 1.        , 1.        ,
        0.5833333 , 0.2       ],
       [0.6666667 , 0.        , 0.        , 0.        , 0.        ,
        0.2       , 0.5       , 0.72727275, 1.        , 0.        ,
        0.5       , 0.        ],
       [0.98      , 0.6393443 , 0.6883117 , 0.43421054, 0.42477876,
        0.5070422 , 0.4651163 , 0.30666667, 0.3409091 , 0.48333332,
        0.30769232, 0.6909091 ],
       [1.        , 0.        , 0.        , 0.        , 0.33333334,
        0.        , 1.        , 0.45454547, 1.        , 1.        ,
        0.75      , 0.        ]], dtype=float32)

In [190]:
dmr_ds['dmr_da_frac'].to_numpy()

array([[0.33333334, 0.18181819, 0.5555556 , 0.        , 0.8       ,
        0.        , 0.9444444 , 0.8947368 , 0.9411765 , 1.        ,
        0.95      , 0.43333334],
       [0.9       , 0.        , 0.33333334, 0.07692308, 0.90909094,
        0.15384616, 0.8888889 , 0.96      , 1.        , 1.        ,
        0.5833333 , 0.2       ],
       [0.6666667 , 0.        , 0.        , 0.        , 0.        ,
        0.2       , 0.5       , 0.72727275, 1.        , 0.        ,
        0.5       , 0.        ],
       [0.98      , 0.6393443 , 0.6883117 , 0.43421054, 0.42477876,
        0.5070422 , 0.4651163 , 0.30666667, 0.3409091 , 0.48333332,
        0.30769232, 0.6909091 ],
       [1.        , 0.        , 0.        , 0.        , 0.33333334,
        0.        , 1.        , 0.45454547, 1.        , 1.        ,
        0.75      , 0.        ]], dtype=float32)

In [184]:
dmr_ds['sample'].to_numpy()

array(['/data2st1/junyi/merged_allcs/PFC/OPC-Oligo',
       '/data2st1/junyi/merged_allcs/PFC/Astro-Epen',
       '/data2st1/junyi/merged_allcs/PFC/Vascular',
       '/data2st1/junyi/merged_allcs/PFC/Neuron',
       '/data2st1/junyi/merged_allcs/PFC/Immune'], dtype='<U43')

In [192]:
dmr_ds['dmr_start'].to_numpy()

array([103641264, 103643359, 103645891, 103651558, 103654369, 103655509,
       103656035, 103656730, 103657027, 103657028, 103658610, 103659079])

In [193]:
dmr_ds['dmr_end'].to_numpy()

array([103641266, 103643361, 103645893, 103651560, 103654371, 103655511,
       103656037, 103656733, 103657029, 103657030, 103658653, 103659264])

In [186]:
dmr_ds['dmr_state'].to_numpy()[1]

array([0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0], dtype=int8)

In [187]:
dmr_ds['dmr_state'].to_numpy()[3]

array([ 1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1], dtype=int8)

In [194]:
dmr_ds['dmr_state'].to_numpy()

array([[-1, -1,  0, -1,  1, -1,  1,  1,  1,  1,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  1,  1,  1,  1,  0,  0],
       [ 0, -1, -1,  0,  0,  0,  0,  0,  1,  0,  0, -1],
       [ 1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1],
       [ 0, -1, -1, -1,  0,  0,  0,  0,  0,  1,  0, -1]], dtype=int8)

In [94]:
dmr_ds['sample'].to_numpy()

array(['/data2st1/junyi/merged_allcs/PFC/OPC-Oligo',
       '/data2st1/junyi/merged_allcs/PFC/Astro-Epen',
       '/data2st1/junyi/merged_allcs/PFC/Vascular',
       '/data2st1/junyi/merged_allcs/PFC/Neuron',
       '/data2st1/junyi/merged_allcs/PFC/Immune'], dtype='<U43')

In [176]:
dmr_ds['dmr_da_frac'].to_numpy()

array([[0.75      ],
       [0.        ],
       [0.25      ],
       [0.10526316],
       [1.        ]], dtype=float32)

In [8]:
#dmr_ds = RegionDS.open(output_dir, region_dim='dms')

<xarray.RegionDS>
Dimensions:       (count_type: 2, dms: 171, sample: 5)
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U12 'chr1-3001834' 'chr1-3003641' ... 'chr1-3098570'
    dms_chrom     (dms) <U4 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_contexts  (dms) <U3 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_p-values  (dms) float64 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_pos       (dms) int64 dask.array<chunksize=(34,), meta=np.ndarray>
  * sample        (sample) <U43 '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo' ...
Data variables:
    dms_da        (sample, dms, count_type) float64 dask.array<chunksize=(1, 34, 1), meta=np.ndarray>
    dms_da_frac   (sample, dms) float64 dask.array<chunksize=(1, 34), meta=np.ndarray>
    dms_residual  (sample, dms) float64 dask.array<chunksize=(1, 34), meta=np.ndarray>
Attributes:
    region_dim:          dms
    region_ds_location:  /data2st1/junyi/merged_allcs/PFC/test_PFC
    chrom_size_path:     /data2st1/junyi/merged_allcs/PFC/test_PFC/chrom_size...

In [38]:
dmr_ds['dms']

<xarray.DataArray 'dms_da' (sample: 5, dms: 171, count_type: 2)>
dask.array<xarray-dms_da, shape=(5, 171, 2), dtype=float64, chunksize=(1, 34, 1), chunktype=numpy.ndarray>
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U12 'chr1-3001834' 'chr1-3003641' ... 'chr1-3098570'
    dms_chrom     (dms) <U4 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_contexts  (dms) <U3 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_p-values  (dms) float64 dask.array<chunksize=(34,), meta=np.ndarray>
    dms_pos       (dms) int64 dask.array<chunksize=(34,), meta=np.ndarray>
  * sample        (sample) <U43 '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo' ...